In [1]:
import sys
import jax
import os
import time
import pickle
import genjax
import glob
import bayes3d as b
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../")
from viz import *
from utils import *
from PIL import Image
from io import BytesIO
import bayes3d.transforms_3d as t3d
from jax.debug import print as jprint
from tqdm import tqdm
from dataclasses import dataclass
from genjax._src.core.pytree.utilities import *
from genjax.generative_functions.distributions import ExactDensity
from tensorflow_probability.substrates import jax as tfp
import jax.tree_util as jtu
from genjax._src.core.transforms.incremental import NoChange, UnknownChange, Diff
console = genjax.pretty()
%matplotlib inline

In [6]:
def normalize_weights(log_weights):
    log_total_weight = logsumexp(log_weights)
    log_normalized_weights = log_weights - log_total_weight
    return log_normalized_weights

def ess(log_normalized_weights):
    log_ess = -logsumexp(2. * log_normalized_weights)
    return jnp.exp(log_ess)


class MCS_Observation:
    def __init__(self, rgb, depth, intrinsics, segmentation):
        """RGBD Image
        
        Args:
            rgb (np.array): RGB image
            depth (np.array): Depth image
            camera_pose (np.array): Camera pose. 4x4 matrix
            intrinsics (b.camera.Intrinsics): Camera intrinsics
            segmentation (np.array): Segmentation image
        """
        self.rgb = rgb
        self.depth = depth
        self.intrinsics = intrinsics
        self.segmentation  = segmentation

load_observations_npz = lambda x : np.load('val7_physics_npzs' + "/{}.npz".format(x),allow_pickle=True)["arr_0"]

def observations_to_data_by_frame(observations, frame_idx, scale = 0.5):
    intrinsics_data = observations[frame_idx].intrinsics
    if 'cam_pose' in dir(observations[frame_idx]):
        cam_pose = observations[frame_idx].cam_pose
    else:
        cam_pose = CAM_POSE_CV2

    intrinsics = b.scale_camera_parameters(b.Intrinsics(intrinsics_data["height"],intrinsics_data["width"],
                            intrinsics_data["fx"], intrinsics_data["fy"],
                            intrinsics_data["cx"],intrinsics_data["cy"],
                            intrinsics_data["near"],intrinsics_data["far"]),scale)
    
    obs = observations[frame_idx]
    depth = np.asarray(jax.image.resize(obs.depth, (int(obs.depth.shape[0] * scale), 
                        int(obs.depth.shape[1] * scale)), 'nearest'))
    seg = np.asarray(jax.image.resize(obs.segmentation, (int(obs.segmentation.shape[0] * scale), 
                        int(obs.segmentation.shape[1] * scale)), 'nearest'))
    rgb = np.asarray(jax.image.resize(obs.rgb, (int(obs.rgb.shape[0] * scale), 
                        int(obs.rgb.shape[1] * scale), 3), 'nearest'))
    gt_image = np.asarray(b.unproject_depth_jit(depth, intrinsics))

    return gt_image, depth, seg, rgb, intrinsics, cam_pose

def observations_to_data(observations, scale = 0.5):
    intrinsics_data = observations[0].intrinsics
    intrinsics = b.scale_camera_parameters(b.Intrinsics(intrinsics_data["height"],intrinsics_data["width"],
                            intrinsics_data["fx"], intrinsics_data["fy"],
                            intrinsics_data["cx"],intrinsics_data["cy"],
                            intrinsics_data["near"],intrinsics_data["far"]),scale)
    
    depths = [jax.image.resize(obs.depth, (int(obs.depth.shape[0] * scale), 
                        int(obs.depth.shape[1] * scale)), 'nearest') for obs in observations]
    segs = [jax.image.resize(obs.segmentation, (int(obs.segmentation.shape[0] * scale), 
                        int(obs.segmentation.shape[1] * scale)), 'nearest') for obs in observations]
    rgbs = [jax.image.resize(obs.rgb, (int(obs.rgb.shape[0] * scale), 
                        int(obs.rgb.shape[1] * scale), 3), 'nearest') for obs in observations]
    
    gt_images = b.unproject_depth_vmap_jit(np.stack(depths), intrinsics)

    return gt_images, depths, segs, rgbs, intrinsics

def fake_masker(point_cloud_image, segmentation, object_mask, object_ids, id, iter):
    object_ids = object_ids.at[iter].set(-1)
    return object_ids, object_mask

def inner_add_mask(object_mask, object_ids, segmentation, id,iter):
    object_mask += (segmentation == id)
    object_ids = object_ids.at[iter].set(id)
    return object_ids, object_mask 

def inner_fake(object_mask, object_ids, segmentation, id,iter):
    object_ids = object_ids.at[iter].set(-1)
    return object_ids, object_mask


def custom_aabb(object_points):
    maxs = jnp.nanmax(object_points, axis = (0,1))
    mins = jnp.nanmin(object_points, axis = (0,1))
    dims = (maxs - mins)
    center = (maxs + mins) / 2
    return dims

@jax.jit
def get_object_mask_debug(point_cloud_image, segmentation):
    segmentation_ids = jnp.unique(segmentation, size = 10, fill_value = -1)
    object_mask = jnp.zeros(point_cloud_image.shape[:2])
    object_ids = jnp.zeros(10)
    def scan_fn(carry, id):
        object_mask, object_ids, iter = carry
        object_ids, object_mask = jax.lax.cond(id == -1, fake_masker, masker_f_debug,*(point_cloud_image, segmentation, object_mask, object_ids, id, iter))
        return (object_mask, object_ids, iter + 1), None
    
    (object_mask, object_ids, _), _ = jax.lax.scan(scan_fn, (object_mask, object_ids, 0), segmentation_ids)
                                               
    object_mask = object_mask > 0
    return object_ids, object_mask




WALL_Z = 14.
CAM_POSE = np.array([[ 1,0,0,0],
[0,0,-1,-4.5], # 4.5 is an arbitrary value
[ 0,1,0,1.5],
[ 0,0,0,1]])
World2Cam = np.linalg.inv(CAM_POSE)
World2Cam[1:3] *= -1
CAM_POSE_CV2 = np.linalg.inv(World2Cam)
cam_pose = CAM_POSE_CV2

def in_camera_view(renderer, known_id, pose):
    """ Check if pose point is in camera view """
    # pose is assumed to be in camera frame
    return jnp.any(b.RENDERER.intrinsics.far != jnp.unique(renderer.render(pose[None,...], jnp.array([known_id]))[...,2]))

@jax.jit
def splice_image(rendered_object_image, obs_image_complement):
    keep_masks = jnp.logical_or(
        (rendered_object_image[...,2] <= obs_image_complement[..., 2]) * 
        rendered_object_image[...,2] > 0.0
        ,
        (obs_image_complement[...,2] == 0)
    )[...,None]
    rendered_images = keep_masks * rendered_object_image + (1.0 - keep_masks) * obs_image_complement
    return rendered_images

@jax.jit
def splice_image_new(rendered_object_image, obs_image_complement, far=150.0):
    keep_masks = jnp.logical_or(
        jnp.logical_and((rendered_object_image[...,2] <= obs_image_complement[..., 2]) * 
        rendered_object_image[...,2] > 0.0, (obs_image_complement[...,2] >= far))
        ,
        (obs_image_complement[...,2] == 0)
    )[...,None]
    rendered_images = keep_masks * rendered_object_image + (1.0 - keep_masks) * obs_image_complement
    return rendered_images

splice_image_vmap = jax.vmap(splice_image, in_axes = (0,None))
splice_image_double_vmap = jax.vmap(splice_image, in_axes = (0,None))



"""
POSSIBLE FAILURE CASES:
1) The object moves so fast that it has nothing to do with the boundary
2) Min dist is not set right, either too small and the obejct misses it OR too large and other objects get counted
3) NOT A BUG but we may have to improve the quality of the object formation
"""

"""
BUGS IDENTIFIED:

Object masking goes wrong in gravity scenes with that stupid pole entering the top ---> need to mae sure this is not 
in the mask

OBJECT MASKING is wrong for frames where object is entering from the top
"""

def preprocess_mcs_physics_scene_debug(observations, MIN_DIST_THRESH = 0.6, scale = 0.1):
    # preprocess object information before running it through model by using the gt_images
    T = len(observations)
    review_stack = []
    init_queue = []
    review_id = 0
    registered_objects = []
    gt_images = []
    gt_images_bg = []
    gt_images_obj = []
    # Rule, for first frame, process any object as object model
    # Afterwards, everything must come from the edges

    get_distance = lambda x,y : np.linalg.norm(x[:3,3]-y[:3,3])
    print("Extracting Meshes")
    obj_pixels = []

    poses = [[] for _ in range(T)]

    for t in tqdm(range(T)):
    # for t in range(T):
        gt_image, depth, seg, rgb, intrinsics, cam_pose = observations_to_data_by_frame(observations, t, scale = 1)
        gt_image = np.asarray(gt_image)
        gt_images.append(gt_image)
        # print("t = ",t)
        seg_ids = np.unique(seg)
        obj_ids_fixed_shape, obj_mask = get_object_mask_debug(gt_image, seg)
        # remove all the -1 indices
        obj_ids = np.delete(np.sort(np.unique(obj_ids_fixed_shape)),0) # This will not be jittable
        # print(obj_ids)
        depth_bg = depth * (1.0 - obj_mask) + intrinsics.far * (obj_mask)
        depth_obj = depth * (obj_mask) + intrinsics.far * (1.0 - obj_mask)
        gt_images_bg.append(np.asarray(b.t3d.unproject_depth(depth_bg, intrinsics)))
        gt_images_obj.append(np.asarray(b.t3d.unproject_depth(depth_obj, intrinsics)))
        obj_pixel_ct = 0

        for obj_id in obj_ids:

            num_pixels = np.sum(seg == obj_id)
            obj_pixel_ct += num_pixels
            point_cloud_segment = gt_image[seg == obj_id]
            dims, pose = b.utils.aabb(point_cloud_segment)
            poses[t].append(pose)
            rows, cols = np.where(seg == obj_id)
            distance_to_edge_1 = min(np.abs(rows - 0).min(), np.abs(rows - intrinsics.height + 1).min())
            distance_to_edge_2 = min(np.abs(cols - 0).min(), np.abs(cols - intrinsics.width + 1).min())
            # print(distance_to_edge_1, distance_to_edge_2)

            if t == 0:
                init_object_model = True
                init_object_model_metadata = {
                    't_init' : t,
                    'pose' : pose,
                    't_full' : t,
                    'full_pose' : pose,
                    'num_pixels' : num_pixels,
                    'mask' : seg == obj_id
                }
                # init_object_model = False
            else:
                init_object_model = False


            if distance_to_edge_1 == 0 or distance_to_edge_2 == 0:
                # check to ensure it is not any object in the review stack or in the init queue
                new_object = True
                if len(review_stack) > 0:
                    # check review stack first
                    distances_rs = [get_distance(pose,r['updating_pose']) for r in review_stack]
                    min_dist = np.min(distances_rs)
                    if min_dist < MIN_DIST_THRESH:
                        new_object = False
                if len(init_queue) > 0:
                    # chec init queue next
                    distances_iq = [get_distance(pose,i['updating_pose']) for i in init_queue]
                    min_dist = np.min(distances_iq)
                    init_queue_idx = distances_iq.index(min_dist)
                    if min_dist < MIN_DIST_THRESH:
                        new_object = False
                if new_object:
                    # Then this is a new object at the boundary not currently accounted for
                    #this means that object is either leaving or entering a scene (at the boundary)
                    review_id += 1
                    print("Adding review")
                    review_stack.append(
                        {
                            'id' : review_id,
                            'num_pixels' : num_pixels,
                            't_init' : t,
                            'distance_to_edge_1' : distance_to_edge_1,
                            'distance_to_edge_2' : distance_to_edge_2,
                            'updating_pose' : pose,
                            'init_pose' : pose
                        }
                    )
            if len(review_stack) > 0:
                # find which object under review is the closest
                distances_rs = [get_distance(pose,r['updating_pose']) for r in review_stack if r['t_init'] == t - 1]

                if len(distances_rs) > 0:
                    min_dist = np.min(distances_rs)
                    review_stack_idx = distances_rs.index(min_dist)
                    if min_dist < MIN_DIST_THRESH:
                        # evaluate if object is moving away or not
                        if num_pixels > review_stack[review_stack_idx]['num_pixels'] or (not (distance_to_edge_1 == 0 or distance_to_edge_2 == 0)):
                            # review passed!
                            # TODO PASS REVIEW
                            init_queue.append(review_stack[review_stack_idx])
                            print("Review passed, added to init queue")
                        else:
                            print("Review failed")
                        del review_stack[review_stack_idx]                 
                    else:
                        # Then this object must not be related to the reviewed object
                        pass
                else:
                    # Then all objects in stack were made in this time step and this object can move on
                    pass

            if len(init_queue) > 0:
                distances_iq = [get_distance(pose,i['updating_pose']) for i in init_queue]
                min_dist = np.min(distances_iq)
                init_queue_idx = distances_iq.index(min_dist)
                if min_dist < MIN_DIST_THRESH:
                    if not (distance_to_edge_1 == 0 or distance_to_edge_2 == 0):
                        # object is now ready to be initialized
                        # print("Obj init")
                        init_object_model = True
                        init_object_model_metadata = {
                            't_init' : init_queue[init_queue_idx]['t_init'],
                            'pose' : init_queue[init_queue_idx]['init_pose'],  # TODO A MORE ACCURATE ESTIMATION OF INIT POSE
                            'full_pose' : pose,
                            't_full' : t,
                            'num_pixels': num_pixels
                        }
                        del init_queue[init_queue_idx]
                    else:
                        # this must be the object but it is still at the boundary, update the pose
                        init_queue[init_queue_idx]['updating_pose'] = pose 
                else:
                    # unrelated object
                    pass


            if init_object_model:
                # This part makes the mesh
                resolution = 0.01
                voxelized = np.rint(point_cloud_segment / resolution).astype(np.int32)
                min_z = voxelized[:,2].min()
                depth_val = voxelized[:,2].max() - voxelized[:,2].min()

                front_face = voxelized[voxelized[:,2] <= min_z+20, :]
                slices = [front_face]
                for i in range(depth_val):
                    slices.append(front_face + np.array([0.0, 0.0, i]))
                full_shape = np.vstack(slices) * resolution

                dims, pose = b.utils.aabb(full_shape)
                # print("before making mesh object", get_gpu_mem())

                mesh = b.utils.make_marching_cubes_mesh_from_point_cloud(
                    b.t3d.apply_transform(full_shape, b.t3d.inverse_pose(pose)),
                    0.075
                )
                # print("before adding", get_gpu_mem())
                # renderer.add_mesh(mesh)
                
                init_object_model_metadata['mesh'] = mesh
                registered_objects.append(init_object_model_metadata)
                print("Adding new mesh for t = {}",init_object_model_metadata['t_init'])
        # Ensure the every review in the review stack has the same time step as the current review
        del_idxs = []
        for i in list(reversed(range(len(review_stack)))):
            if review_stack[i]['t_init'] < t:
                print("Review Stack not resolved, object may have left view, deleting review")
                del review_stack[i]

        if len(review_stack) > 0 and (not np.all([r['t_init'] == t for r in review_stack])):
            print("REVIEW STACK HAS NOT BEEN FULLY RESOLVED, LOGIC ERROR")

        obj_pixels.append(obj_pixel_ct)

    gt_images = np.stack(gt_images)
    gt_images_bg = np.stack(gt_images_bg)
    gt_images_obj = np.stack(gt_images_obj)

    # now extract the data at low resolution
    print("Extracting downsampled data")
    gt_images_downsampled, _, _, _, intrinsics_downsampled = observations_to_data(observations,scale = scale)
    # get new height and width
    new_h = intrinsics_downsampled.height
    new_w = intrinsics_downsampled.width
    # resize the depths
    depths_bg = [jax.image.resize(x[...,2], (new_h, new_w), 'nearest') for x in gt_images_bg]
    depths_obj = [jax.image.resize(x[...,2], (new_h, new_w), 'nearest') for x in gt_images_obj]
    # get new point clouds based on new intrisics
    gt_images_bg_downsampled = jnp.stack([b.t3d.unproject_depth(x, intrinsics_downsampled) for x in depths_bg])
    gt_images_obj_downsampled = jnp.stack([b.t3d.unproject_depth(x, intrinsics_downsampled) for x in depths_obj])

    # hack to determine gravity scene
    is_gravity = len(init_queue) > 0

    return (gt_images_downsampled,gt_images_bg_downsampled,gt_images_obj_downsampled,intrinsics_downsampled),(gt_images, gt_images_bg, gt_images_obj,intrinsics), registered_objects, obj_pixels, is_gravity, poses, cam_pose


def masker_f_debug(point_cloud_image, segmentation, object_mask, object_ids, id, iter):

    mask = segmentation == id
    # Use the mask to select elements, keeping the original shape
    masked_point_cloud_segment = jnp.where(mask[..., None], point_cloud_image, jnp.nan)

    bbox_dims = custom_aabb(masked_point_cloud_segment)
    # jprint("{}", bbox_dims)
    is_occluder = jnp.logical_or(jnp.logical_or(jnp.logical_or(jnp.logical_or(
                    (bbox_dims[0] < 0.1),
                    (bbox_dims[1] < 0.1)),
                (bbox_dims[1] + bbox_dims[0] > 2.4)),
            # (bbox_dims[0] > 1.3)),
            False),
        (bbox_dims[2] > 1.1)
    )
    return jax.lax.cond(is_occluder, inner_fake, inner_add_mask,*(object_mask, object_ids, segmentation, id, iter))


In [5]:
# Loading and preprocessing all data and renderer
SCALE = 0.2
paths = "/home/arijitdasgupta/bayes3d/scripts/experiments/intphys/mcs/val7_physics_npzs/*npz"
files = sorted(glob.glob(paths))
num_scenes = len(files)
for i in tqdm(range(0,num_scenes)):
    scene_name = files[i].split("/")[-1][:-4]
    # with open(f"/home/arijitdasgupta/bayes3d/scripts/experiments/intphys/mcs/pickled_data/{scene_name}.pkl", 'rb') as file:
    #     preprocessed_data = pickle.load(file)
    observations = load_observations_npz(scene_name)
    preprocessed_data = preprocess_mcs_physics_scene_debug(observations, MIN_DIST_THRESH=0.6, scale=SCALE)
    (gt_images, gt_images_bg, gt_images_obj, intrinsics),\
    (gt_images_orig, gt_images_bg_orig, gt_images_obj_orig, intrinsics_orig),\
    registered_objects, obj_pixels, is_gravity, poses, cam_pose = preprocessed_data
    inverse_cam_pose = jnp.linalg.inv(CAM_POSE_CV2)
    print(scene_name)
    display(display_video([b.get_depth_image(gt_images_bg_orig[i,...,2]) for i in range(gt_images.shape[0])]))
    break
    

  0%|          | 0/62 [00:00<?, ?it/s]

Extracting Meshes


t =  0


[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[16.887222  7.104474  0.      ]
[0.9902556  1.2005881  0.74014854]
Adding new mesh for t = {} 0
t =  1
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
t =  2
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]


t =  3
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
t =  4
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
t =  5
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
t =  6
[16.887222  7.104474  0.      ]
[0.9902556  1.2005881  0.74014854]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
t =  7
[0.9902556  1.2005881  0.74014854]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[16.887222  7.104474  0.      ]
t =  8
[0.9902556  1.2005881  0.74014854]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[8.1917173e-01 2.4175644e-04 2.5612831e-01]
[16.887222  7.104474  0.      ]


...

t =  93
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.8851359  0.87830764 0.4465661 ]
t =  94
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.9902556  1.2005881  0.74014854]
[0.8851359  0.87830764 0.4465661 ]
[16.887222  7.104474  0.      ]
t =  95
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.9902556  1.2005881  0.74014854]
[0.8851359  0.87830764 0.4465661 ]
[16.887222  7.104474  0.      ]
t =  96
[0.9902556  1.2005881  0.74014854]
[0.8851359  0.87830764 0.4465661 ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[16.887222  7.104474  0.      ]
t =  97
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
[0.8851359  0.87830764 0.4465661 ]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]


100%|██████████| 100/100 [00:06<00:00, 15.99it/s]


t =  98
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.8851359  0.87830764 0.4465661 ]
[0.9902556  1.2005881  0.74014854]
[16.887222  7.104474  0.      ]
t =  99
[0.9902556  1.2005881  0.74014854]
[1.6793526e+01 1.0316491e-02 1.0552001e+01]
[0.8851359  0.87830764 0.4465661 ]
[16.887222  7.104474  0.      ]
Extracting downsampled data
passive_physics_validation_gravity_support_0001_01


<IPython.core.display.HTML object>

  0%|          | 0/62 [00:11<?, ?it/s]


In [ ]:
bbbox = registered_objects[0]['mesh'].bounding_box.bounds
dimensions = bbbox[1] - bbbox[0]
dimensions